In [123]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from typing import List
from numpy.typing import NDArray

In [124]:
data = load_iris()
x = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.3)

In [125]:

class OWA:
    classifiers: List
    classes: List
    def __init__(self, classifiers) -> None:
        self.classifiers = classifiers
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        for c in self.classifiers:
            c.fit(X, y)
    
    def DP(self, x_test, weights=[]):
        weights = [1/len(self.classifiers) for _ in range(len(self.classifiers))] if not sum(weights)==1 or weights==[] else weights
        dp = []
        dpw = []
        for c in self.classifiers:
            dp.append(c.predict_proba(x_test))
        dp = np.array(dp).reshape(len(self.classifiers), len(self.classes))
        dpw = [weights @ dp.T[i] for i in range(len(self.classes))]
        return dp, dpw
    
    def predict(self, datas:NDArray, weights=[]):
        powa = []
        for data in datas:
            _, dpw=self.DP(data.reshape(1, -1), weights=weights)
            powa.append(np.argmax(dpw))
        return np.array(powa)
    
    def classifiers_predict(self, X_test):
        p = []
        for c in self.classifiers:
            p.append(c.predict(X_test))
        return p
        

In [126]:
from numpy import argmax


classifiers = [LogisticRegression(max_iter=10000), MultinomialNB(), KNeighborsClassifier(), GaussianNB()]
owa = OWA(classifiers)
owa.fit(X_train, y_train)

print("TESTING")
dp, dpw = owa.DP(X_test[0].reshape(1, -1))
print(f"true value:{y_test[0]}")
print(f"Decision profile:\n{dp}")
for row in dp:
    print(f"arg max of decision profile of row{row}:{np.argmax(row)}")
print(f"decision profile product to weights:{dpw}")
print(f"arg max of decision profile producted:{np.argmax(dpw)}")
print("TESTING")

pre_owa = owa.predict(datas=X_test)
print(accuracy_score(y_test, pre_owa))
pre_cls = owa.classifiers_predict(X_test)
for i in pre_cls:
    print(accuracy_score(y_test, i))

TESTING
true value:0
Decision profile:
[[9.71678353e-01 2.83213920e-02 2.54676920e-07]
 [6.74281881e-01 1.94688696e-01 1.31029424e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 2.79175514e-22 2.73780288e-27]]
arg max of decision profile of row[9.71678353e-01 2.83213920e-02 2.54676920e-07]:0
arg max of decision profile of row[0.67428188 0.1946887  0.13102942]:0
arg max of decision profile of row[1. 0. 0.]:0
arg max of decision profile of row[1.00000000e+00 2.79175514e-22 2.73780288e-27]:0
decision profile product to weights:[0.911490058458147, 0.05575252199532582, 0.03275741954652735]
arg max of decision profile producted:0
TESTING
0.9333333333333333
0.9333333333333333
0.7777777777777778
0.9777777777777777
0.9111111111111111
